In [1]:
from pickle5 import pickle
import pandas as pd

r = pd.read_csv('data/ratings.csv')
r.tail()

,user_id,book_id,rating
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5
5976478,49925,1023,4


In [2]:
pkl_filename = "SVDmodel.pkl"
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

print("model was successfully loaded")

model was successfully loaded


In [3]:
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
from random import sample

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(r[['user_id', 'book_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.15)

In [4]:
test_example = sample(testset, 1)
print(test_example)

[(37872, 2731, 4.0)]


In [12]:
results = pickle_model.test(test_example)
print(results)

[Prediction(uid=37872, iid=2731, r_ui=4.0, est=3.7505202619100673, details={'was_impossible': False})]
